In [ ]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.14 s


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/My Drive/regression/'
os.chdir(root_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 24.7 s


In [ ]:
import csv
import pandas as pd
import numpy as np

from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

time: 1.31 s


In [ ]:
data_df = pd.read_csv("main_epochdetails.csv")
data_df.head()

,activation,batch_size,best_acc,best_loss,best_train_acc,best_train_loss,filter1,kernelsize_1,filter2,kernelsize_2,filter3,kernelsize_3,hidden_1,hidden_2,learning_rate,momentum,optimizer,p_reduct,step_1_test_acc,step_1_test_loss,step_2_test_acc,step_2_test_loss,step_3_test_acc,step_3_test_loss,step_4_test_acc,step_4_test_loss,step_5_test_acc,step_5_test_loss,step_6_test_acc,step_6_test_loss,step_7_test_acc,step_7_test_loss,step_8_test_acc,step_8_test_loss,step_9_test_acc,step_9_test_loss,step_10_test_acc,step_10_test_loss,step_11_test_acc,step_11_test_loss,...,step_31_test_acc,step_31_test_loss,step_32_test_acc,step_32_test_loss,step_33_test_acc,step_33_test_loss,step_34_test_acc,step_34_test_loss,step_35_test_acc,step_35_test_loss,step_36_test_acc,step_36_test_loss,step_37_test_acc,step_37_test_loss,step_38_test_acc,step_38_test_loss,step_39_test_acc,step_39_test_loss,step_40_test_acc,step_40_test_loss,step_41_test_acc,step_41_test_loss,step_42_test_acc,step_42_test_loss,step_43_test_acc,step_43_test_loss,step_44_test_acc,step_44_test_loss,step_45_test_acc,step_45_test_loss,step_46_test_acc,step_46_test_loss,step_47_test_acc,step_47_test_loss,step_48_test_acc,step_48_test_loss,step_49_test_acc,step_49_test_loss,step_50_test_acc,step_50_test_loss
0,relu,64,0.7965,0.595114,0.728682,0.764486,96,5,96,3,96,4,100,125,0.0015,0.95,adam,0,0.4229,1.599571,0.5202,1.341725,0.5530,1.284754,0.5807,1.189023,0.5654,1.229672,0.6130,1.124784,0.6011,1.223553,0.6038,1.172521,0.7076,0.840214,0.6967,0.879995,0.6906,0.902951,...,0.7143,0.847992,0.7804,0.632164,0.7727,0.650877,0.7241,0.792020,0.7649,0.671802,0.7621,0.679266,0.7443,0.732199,0.6896,0.949306,0.7426,0.762334,0.7867,0.617402,0.7536,0.717102,0.7589,0.704546,0.7815,0.639940,0.7329,0.799030,0.7778,0.650016,0.7965,0.595114,0.7559,0.707896,0.7782,0.652541,0.7560,0.704031,0.6551,1.079705
1,relu,64,0.8126,0.549769,0.760176,0.676151,64,5,128,5,128,5,100,100,0.0015,0.99,adam,0,0.4998,1.390152,0.5883,1.153208,0.5760,1.249412,0.6396,1.007560,0.5116,1.411761,0.6838,0.886597,0.6262,1.044519,0.7069,0.842905,0.5749,1.308798,0.7458,0.730744,0.6456,1.027054,...,0.7923,0.611627,0.7448,0.759005,0.7711,0.683138,0.7752,0.679417,0.8103,0.561076,0.7754,0.670295,0.7866,0.652615,0.8004,0.593488,0.8074,0.564146,0.7784,0.661562,0.8093,0.562898,0.8126,0.549769,0.7348,0.832646,0.7709,0.679137,0.8093,0.566845,0.7609,0.739477,0.7961,0.611452,0.7995,0.610755,0.7929,0.613329,0.7896,0.642212
2,relu,64,0.7886,0.615561,0.729345,0.762990,64,4,64,4,128,4,100,125,0.0015,0.95,adam,0,0.4640,1.458794,0.5537,1.235074,0.5157,1.324192,0.6312,1.023915,0.5717,1.233983,0.5945,1.142362,0.6798,0.906518,0.6753,0.902154,0.6815,0.880340,0.6360,1.045974,0.6937,0.863215,...,0.7768,0.653406,0.7724,0.665004,0.7445,0.753315,0.7825,0.632390,0.7839,0.629152,0.7773,0.665828,0.7351,0.792072,0.7772,0.656150,0.7701,0.684644,0.7733,0.669080,0.7706,0.671042,0.7814,0.646684,0.7886,0.615561,0.7656,0.689567,0.7249,0.828165,0.7800,0.662457,0.7778,0.657332,0.7838,0.648575,0.7849,0.632220,0.7768,0.660482
3,relu,32,0.8024,0.577942,0.752614,0.696140,64,5,128,5,96,4,125,125,0.0010,0.90,adam,0,0.5104,1.356928,0.3385,2.233133,0.5496,1.272580,0.5861,1.214216,0.6459,0.995530,0.6525,0.971852,0.6809,0.892278,0.7123,0.802811,0.6804,0.899192,0.7406,0.744694,0.7443,0.734993,...,0.7757,0.663841,0.7374,0.784743,0.7971,0.597761,0.7777,0.668055,0.7862,0.620247,0.7641,0.699347,0.7837,0.636170,0.7856,0.646041,0.7989,0.588448,0.7963,0.595905,0.7653,0.677547,0.7845,0.632123,0.7981,0.583027,0.7671,0.693960,0.7864,0.636048,0.7645,0.701268,0.8002,0.581245,0.8024,0.577942,0.8025,0.586200,0.7976,0.594688
4,relu,32,0.8044,0.576930,0.744993,0.723093,96,5,128,4,128,5,125,125,0.0020,0.95,adam,0,0.4452,1.520592,0.5176,1.407517,0.6096,1.111541,0.6536,1.003592,0.5837,1.210475,0.6517,1.014945,0.6182,1.088522,0.6946,0.862992,0.6879,0.921019,0.6712,0.961724,0.7275,0.801297,...,0.7768,0.659099,0.7349,0.799387,0.7946,0.609357,0.7971,0.582174,0.7955,0.603193,0.7845,0.640911,0.7963,0.593850,0.786

time: 1.25 s


In [ ]:
col_list = [
            "activation", "batch_size", "best_acc", "best_loss", "best_train_acc", "best_train_loss", 
            "filter1", "filter2", "filter3", "hidden_1", "hidden_2", "kernelsize_1", 
            "kernelsize_2", "kernelsize_3", "learning_rate", "momentum", "optimizer", "p_reduct"
] + [f"step_{i}_test_acc" for i in range(1,51)]

time: 2.48 ms


In [ ]:
clean_df = data_df[col_list].copy()
clean_df.head()

,activation,batch_size,best_acc,best_loss,best_train_acc,best_train_loss,filter1,filter2,filter3,hidden_1,hidden_2,kernelsize_1,kernelsize_2,kernelsize_3,learning_rate,momentum,optimizer,p_reduct,step_1_test_acc,step_2_test_acc,step_3_test_acc,step_4_test_acc,step_5_test_acc,step_6_test_acc,step_7_test_acc,step_8_test_acc,step_9_test_acc,step_10_test_acc,step_11_test_acc,step_12_test_acc,step_13_test_acc,step_14_test_acc,step_15_test_acc,step_16_test_acc,step_17_test_acc,step_18_test_acc,step_19_test_acc,step_20_test_acc,step_21_test_acc,step_22_test_acc,step_23_test_acc,step_24_test_acc,step_25_test_acc,step_26_test_acc,step_27_test_acc,step_28_test_acc,step_29_test_acc,step_30_test_acc,step_31_test_acc,step_32_test_acc,step_33_test_acc,step_34_test_acc,step_35_test_acc,step_36_test_acc,step_37_test_acc,step_38_test_acc,step_39_test_acc,step_40_test_acc,step_41_test_acc,step_42_test_acc,step_43_test_acc,step_44_test_acc,step_45_test_acc,step_46_test_acc,step_47_test_acc,step_48_test_acc,step_49_test_acc,step_50_test_acc
0,relu,64,0.7965,0.595114,0.728682,0.764486,96,96,96,100,125,5,3,4,0.0015,0.95,adam,0,0.4229,0.5202,0.5530,0.5807,0.5654,0.6130,0.6011,0.6038,0.7076,0.6967,0.6906,0.5593,0.6972,0.6660,0.7177,0.7330,0.7554,0.7412,0.6562,0.7483,0.7098,0.7584,0.7546,0.6989,0.7402,0.7536,0.7474,0.7665,0.7512,0.7769,0.7143,0.7804,0.7727,0.7241,0.7649,0.7621,0.7443,0.6896,0.7426,0.7867,0.7536,0.7589,0.7815,0.7329,0.7778,0.7965,0.7559,0.7782,0.7560,0.6551
1,relu,64,0.8126,0.549769,0.760176,0.676151,64,128,128,100,100,5,5,5,0.0015,0.99,adam,0,0.4998,0.5883,0.5760,0.6396,0.5116,0.6838,0.6262,0.7069,0.5749,0.7458,0.6456,0.7231,0.7412,0.7522,0.7369,0.7678,0.7296,0.7677,0.7499,0.7719,0.7318,0.7142,0.7555,0.7569,0.7825,0.7517,0.8088,0.7938,0.7777,0.7896,0.7923,0.7448,0.7711,0.7752,0.8103,0.7754,0.7866,0.8004,0.8074,0.7784,0.8093,0.8126,0.7348,0.7709,0.8093,0.7609,0.7961,0.7995,0.7929,0.7896
2,relu,64,0.7886,0.615561,0.729345,0.762990,64,64,128,100,125,4,4,4,0.0015,0.95,adam,0,0.4640,0.5537,0.5157,0.6312,0.5717,0.5945,0.6798,0.6753,0.6815,0.6360,0.6937,0.6110,0.6460,0.5507,0.7472,0.6999,0.7434,0.7252,0.7306,0.7428,0.6890,0.6959,0.7662,0.7641,0.7615,0.7388,0.7244,0.7281,0.7593,0.7304,0.7768,0.7724,0.7445,0.7825,0.7839,0.7773,0.7351,0.7772,0.7701,0.7733,0.7706,0.7814,0.7886,0.7656,0.7249,0.7800,0.7778,0.7838,0.7849,0.7768
3,relu,32,0.8024,0.577942,0.752614,0.696140,64,128,96,125,125,5,5,4,0.0010,0.90,adam,0,0.5104,0.3385,0.5496,0.5861,0.6459,0.6525,0.6809,0.7123,0.6804,0.7406,0.7443,0.7027,0.6995,0.7429,0.6997,0.6901,0.7679,0.7333,0.7602,0.7572,0.7439,0.7488,0.7769,0.7605,0.7750,0.7877,0.7430,0.7816,0.7200,0.7666,0.7757,0.7374,0.7971,0.7777,0.7862,0.7641,0.7837,0.7856,0.7989,0.7963,0.7653,0.7845,0.7981,0.7671,0.7864,0.7645,0.8002,0.8024,0.8025,0.7976
4,relu,32,0.8044,0.576930,0.744993,0.723093,96,128,128,125,125,5,4,5,0.0020,0.95,adam,0,0.4452,0.5176,0.6096,0.6536,0.5837,0.6517,0.6182,0.6946,0.6879,0.6712,0.7275,0.7434,0.7408,0.7096,0.7204,0.7043,0.7660,0.7630,0.7589,0.7699,0.7462,0.7617,0.7085,0.7467,0.7551,0.7846,0.7346,0.7591,0.7774,0.7620,0.7768,0.7349,0.7946,0.7971,0.7955,0.7845,0.7963,0.7865,0.7625,0.7580,0.7843,0.8044,0.7982,0.7866,0.8030,0.7847,0.7772,0.7893,0.7980,0.8070


time: 152 ms


In [ ]:
from tqdm import tqdm_notebook
#from tqdm import tqdm_notebook as tqdm
def time_series_to_supervised(df, in_size=5):
  """ Converting our time series data from epoch 1-50, into 
      supervised time series
  """

  hyper_cols = [
          "activation", "batch_size", "best_acc", "best_loss", "best_train_acc", "best_train_loss", 
            "filter1", "filter2", "filter3", "hidden_1", "hidden_2", "kernelsize_1", 
            "kernelsize_2", "kernelsize_3", "learning_rate", "momentum", "optimizer", "p_reduct"]
    
  main_cols = [ "epoch", "input_1", "input_2", "input_3", "input_4", "input_5", "output" ]
  df_main_dataset = pd.DataFrame(columns=[hyper_cols+main_cols])

  tmp_val_ = np.transpose(df[[f"step_{i}_test_acc" for i in range(1,51)]].values)
  for i in range(5): tmp_val_ = np.insert(tmp_val_, 0,  [None for cnt in range(tmp_val_.shape[1])], axis=0)
  t_val = np.transpose(tmp_val_)
  for idx in tqdm_notebook(range(len(t_val))):
    tmp_df_ = pd.DataFrame(t_val[idx], columns=["loss_acc"])
    for i in range(50):
      tmp_df_main_ = pd.DataFrame(np.array([np.insert(np.transpose(tmp_df_.shift(-i)[:in_size+1].values), 0 , i+1)]), columns=[main_cols])
      for h in hyper_cols:
        tmp_df_main_[h] = df[h].iloc[idx]
      df_main_dataset = df_main_dataset.append(tmp_df_main_, ignore_index=True)
  #df_main_dataset["epoch"] = df_main_dataset["epoch"].astype('int')
  return df_main_dataset

time: 23.1 ms


In [ ]:
result_df = time_series_to_supervised(clean_df)
result_df.to_csv("timeseries_dataset.csv", index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



time: 32min 45s


In [ ]:
result_df.head()

,activation,batch_size,best_acc,best_loss,best_train_acc,best_train_loss,epoch,filter1,filter2,filter3,hidden_1,hidden_2,input_1,input_2,input_3,input_4,input_5,kernelsize_1,kernelsize_2,kernelsize_3,learning_rate,momentum,optimizer,output,p_reduct
0,relu,64,0.7965,0.595114,0.728682,0.764486,1.0,96,96,96,100,125,NaN,NaN,NaN,NaN,NaN,5,3,4,0.0015,0.95,adam,0.4229,0
1,relu,64,0.7965,0.595114,0.728682,0.764486,2.0,96,96,96,100,125,NaN,NaN,NaN,NaN,0.4229,5,3,4,0.0015,0.95,adam,0.5202,0
2,relu,64,0.7965,0.595114,0.728682,0.764486,3.0,96,96,96,100,125,NaN,NaN,NaN,0.4229,0.5202,5,3,4,0.0015,0.95,adam,0.5530,0
3,relu,64,0.7965,0.595114,0.728682,0.764486,4.0,96,96,96,100,125,NaN,NaN,0.4229,0.5202,0.5530,5,3,4,0.0015,0.95,adam,0.5807,0
4,relu,64,0.7965,0.595114,0.728682,0.764486,5.0,96,96,96,100,125,NaN,0.4229,0.5202,0.5530,0.5807,5,3,4,0.0015,0.95,adam,0.5654,0


time: 69.6 ms


In [ ]:
# np.transpose(clean_df[[f"step_{i}_test_acc" for i in range(1,51)]].values)

time: 863 µs


In [ ]:
# tmp_val_ = np.transpose(clean_df[[f"step_{i}_test_acc" for i in range(1,51)]].values)
# for i in range(5): tmp_val_ = np.insert(tmp_val_, 0,  [None for cnt in range(323)], axis=0)
# tmp_val_

time: 967 µs


In [ ]:
# np.transpose(tmp_val_)[0]

time: 713 µs


In [ ]:
# tmp_df_ = pd.DataFrame(np.transpose(tmp_val_)[0], columns=["loss_acc"])
# tmp_df_.head(10)

time: 814 µs


In [ ]:
# hyper_cols = [
#             "activation", "batch_size", "best_acc", "best_loss", "best_train_acc", "best_train_loss", 
#             "filters1", "filters2", "filters3", "full_hidd1", "full_hidd2", "ksize1", 
#             "ksize2", "ksize3", "learning_rate", "momentum", "optimizer", "patch_reduction"
# ]
# main_cols = [ "epoch", "input_1", "input_2", "input_3", "input_4", "input_5", "output" ]
# df_main_dataset = pd.DataFrame(columns=[hyper_cols+main_cols])
# for i in range(50):
#   tmp_df_main_ = pd.DataFrame(np.array([np.insert(np.transpose(tmp_df_.shift(-i)[:6].values), 0 , i+1)]), columns=[main_cols])
#   df_main_dataset = df_main_dataset.append(tmp_df_main_, ignore_index = True)
# df_main_dataset[hyper_cols] = [clean_df[hyper_cols].iloc[0].values for i in range(50)]
# df_main_dataset["epoch"] = df_main_dataset["epoch"].astype('int')

time: 8.1 ms


In [ ]:
# df_main_dataset.head()

time: 1.45 ms
